# Sample CloudSql Connection

# Need to set up for each instance - Add your VM's IP to the Authorized Networks
**1. Find the external IP of your JupyterLab VM**
https://console.cloud.google.com/compute/instances?project=adsp-34002-on02-sopho-scribe&authuser=1
 * Go to VM Instances
 * Find your JupyterLab VM
 * Copy the External IP address (looks like 34.91.100.45).

**2. Add the VM's IP to your Cloud SQL authorized networks**
https://console.cloud.google.com/sql/instances/currensee-sql/connections/networking?authuser=1&project=adsp-34002-on02-sopho-scribe
* Go to Cloud SQL instances. 
* Click your instance.
* Click Connections in the left sidebar.
* Scroll to Authorized networks → Add network.
* Name: anything like jupyterlab-vm
* Network: paste the external IP you just copied (e.g., 34.91.100.45/32)
* IMPORTANT: Add /32 to allow only that single IP.
* Click Save.

It will take ~30 seconds to update.

In [1]:
#%pip install -q google-cloud-secret-manager==2.23.3
#%pip install -q SQLAlchemy==2.0.40

In [2]:
#%pip install psycopg2-binary sqlalchemy pandas

In [1]:
from google.cloud import secretmanager
import pandas as pd
from sqlalchemy import create_engine, text

#### Retrieve credentials from Secret Manager

In [39]:
def access_secret(secret_id):
    client = secretmanager.SecretManagerServiceClient()
    name = f"projects/{PROJECT_ID}/secrets/{secret_id}/versions/latest"
    response = client.access_secret_version(name=name)
    return response.payload.data.decode("UTF-8")

#### Configuration

In [40]:
PROJECT_ID = 'adsp-34002-on02-sopho-scribe'
REGION = 'us-central1'
DB_NAME = 'crm'
#DB_NAME = 'postgres'
DB_USER = access_secret('cloudSqlUser')
DB_PASSWORD = access_secret('cloudSqlUserPassword')
DB_HOST = '35.232.92.211'
DB_PORT = '5432'

#### Create SQLAlchemy engine

In [41]:
engine = create_engine(
    f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}',
    connect_args={'sslmode': 'require'}
)

In [48]:
df_result = pd.read_sql("SELECT * FROM Employees limit 10", con=engine)
print(df_result)

                             EmployeeID FirstName    LastName  \
0  389300f0-5cac-4a60-ac26-e0d633027e4f      Jane  Moneypenny   
1  4e87f0b4-7467-40ab-90bb-66eabd26b260   Michael      Harmon   
2  e27b912b-579e-43b3-b665-26b4ca809a2c       Joe      Thomas   
3  e5907e26-1a3b-4b4d-bf29-b5ad343f7999     Bryan        Mann   
4  fd584f2d-a377-4968-ad58-34f60d97d88e  Jonathan       Mason   
5  43ee5ab1-cbed-4591-be68-1c16849acdda      Ruth     Aguilar   
6  0aee491b-76c8-498e-8fb8-4b51294b81ea    Robert          Yu   
7  539174fc-3a3f-44ed-a352-567ff253f406   Phillip         Lee   
8  2cc47b60-9ff4-4982-96cb-b14eaaa46cab      Lisa       Jones   
9  f96042cc-1a96-4de9-9c36-43ad95a7bb1b      Tara      Foster   

                         Title                          Email  \
0         Relationship Manager  jane.moneypenny1@bankwell.com   
1  Senior Relationship Manager    michael.harmon@bankwell.com   
2           Product Specialist        joe.thomas@bankwell.com   
3  Senior Relationship M

# Generate Fake Data

In [5]:
%pip install faker

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import random
import os
import re
from faker import Faker

# Initialize Faker instance
#Faker is a Python package that integrates fake data for you.

#Some hard coded data of publicly traded companies to be our mock clients


In [7]:

fake = Faker()

# Helper function to generate synthetic employee data
def generate_employee_data(num_employees=10, ourcompany_name = 'bankwell'):
    Company = ourcompany_name
    employees = [
        {
            'EmployeeID': fake.unique.uuid4(),
            'FirstName': "Jane",
            'LastName': "Moneypenny",
            'Title': "Relationship Manager",
            'Email': "jane.moneypenny1@bankwell.com",
            'Phone': fake.phone_number(),
            'HireDate': fake.date_this_decade(),
            'Department': 'Enterprise Investment',
            'Market': 'San Fransisco'
        }
    ]

    for _ in range(num_employees-1):
        EmployeeID = fake.unique.uuid4()
        FirstName = fake.first_name()
        LastName = fake.last_name()
        Title = random.choice(['Finance Assistant', 'Financial Advisor', 'Senior Relationship Manager', 'Product Specialist', 'Relationship Manager'])
        Phone = fake.phone_number()
        Department = random.choice(['Enterprise Investment', 'Small Business Investment', 'Operations', 'Sales', 'Customer Support'])
        HireDate = fake.date_this_decade()
        Market = random.choice(['San Fransisco', 'New York City', 'Boston', 'Denver', 'Los Angeles', 'Miami', 'Washington DC', 'Seattle', 'Dallas', 'Chicago'])
        Company_clean = re.sub(r'\W+', '', Company).lower()
        Email = f"{FirstName.lower()}.{LastName.lower()}@{Company_clean}.com"


        employees.append({
            'EmployeeID': EmployeeID,
            'FirstName': FirstName,
            'LastName': LastName,
            'Title': Title,
            'Email': Email,
            'Phone': Phone,
            'HireDate': HireDate,
            'Department': Department,
            'Market': Market
        })
    return pd.DataFrame(employees)

# Helper function to generate point of contact and info for a Company
def generate_point_of_contact(Company_name):
    AccountID = fake.unique.uuid4()
    FirstName = fake.first_name()
    LastName = fake.last_name()
    ContactTitle = random.choice(["Senior Director", "Manager", "Director", "VP", "Consultant"])
    Phone = fake.phone_number()
    Website = fake.url()
    Location = random.choice(['San Fransisco', 'New York City', 'Boston', 'Denver', 'Los Angeles', 'Miami', 'Washington DC', 'Seattle', 'Dallas', 'Chicago'])
    AnnualRevenue = random.randint(1000000, 50000000)
    TotalAccountBal = random.randint(1000000, 50000000)

    # Clean Company name for use in email
    Company_clean = re.sub(r'\W+', '', Company_name).lower()
    Email = f"{FirstName.lower()}.{LastName.lower()}@{Company_clean}.com"

    return {
        "AccountID": AccountID,
        "ContactFirstName": FirstName,
        "ContactLastName": LastName,
        "ContactTitle": ContactTitle,
        "Phone": Phone,
        "Email": Email,
        "Website": Website,
        "Location": Location,
        "AnnualRevenue": AnnualRevenue,
        "TotalAccountBal": TotalAccountBal
    }


def generate_account_data(num_accounts=5, clients_company = []):
# Build the data
    accounts = []
    for company in clients_company:
        contact = generate_point_of_contact(company["Company"])
        record = {
          "Company": company["Company"],
          "industry": company["industry"],
          **contact
      }
        accounts.append(record)
    return pd.DataFrame(accounts)




# Helper function to generate synthetic opportunity data
def generate_opportunity_data(accounts_df, num_opportunities_per_account=3):
    opportunities = []
    for _, account in accounts_df.iterrows():
        num_opportunities = random.randint(1, num_opportunities_per_account)
        for _ in range(num_opportunities):
            opportunities.append({
                'OpportunityID': fake.unique.uuid4(),
                'AccountID': account['AccountID'],
                'OpportunityName': fake.bs(),
                'Stage': random.choice(['Prospecting', 'Qualification', 'Proposal', 'Negotiation', 'Won', 'Lost', 'Closed']),
                'Type': random.choice(['New Business', 'Existing Business', 'Renewal', 'Upsell']),
                'CloseDate': fake.date_this_year(),
                'Amount': random.randint(50000, 500000),
            })
    return pd.DataFrame(opportunities)

# Helper function to generate synthetic employee-contact relationship data
def generate_employee_contact_data(employees_df, accounts_df, num_relationships_per_employee=2):
    relationships = []
    for _, employee in employees_df.iterrows():
        num_relationships = random.randint(1, num_relationships_per_employee)
        for _ in range(num_relationships):
            account = random.choice(accounts_df['AccountID'].tolist())
            relationships.append({
                'EmployeeID': employee['EmployeeID'],
                'EmployeeFirstName': employee['FirstName'],
                'EmployeeLastName': employee['LastName'],
                'AccountID': account,
                'Company': accounts_df.loc[accounts_df['AccountID'] == account, 'Company'].iloc[0],
                'Industry': accounts_df.loc[accounts_df['AccountID'] == account, 'industry'].iloc[0],
                'ContactFirstName': accounts_df.loc[accounts_df['AccountID'] == account, 'ContactFirstName'].iloc[0],
                'ContactLastName': accounts_df.loc[accounts_df['AccountID'] == account, 'ContactLastName'].iloc[0],
                'ContactEmail': accounts_df.loc[accounts_df['AccountID'] == account, 'Email'].iloc[0],
                'ContactTitle': accounts_df.loc[accounts_df['AccountID'] == account, 'ContactTitle'].iloc[0],
                'ContactPhone': accounts_df.loc[accounts_df['AccountID'] == account, 'Phone'].iloc[0],
            })
    return pd.DataFrame(relationships)


def generate_portfolios(df_accounts, max_positions=10, instruments=[]):
    portfolio_records = []

    for _, row in df_accounts.iterrows():
        AccountId = row["AccountID"]
        Company = row["Company"]
        num_positions = random.randint(5, 10)
        positions = random.sample(instruments, num_positions)
        for symbol, instrument_type in positions:
            portfolio_records.append({
                "AccountID": AccountId,
                "Company_name": Company,
                "symbol": symbol,
                "instrument_type": instrument_type
            })

    return pd.DataFrame(portfolio_records)



In [43]:
# Right now companies can only be selected from this list of publicly traded companies 
# Likely want this to be random instead..
clients_company_info = [
        {"Company": "Broadcom", "industry": "Technology"},
        {"Company": "Cisco", "industry": "Technology"},
        {"Company": "Palantir Technologies", "industry": "Technology"},
        {"Company": "Fiserv", "industry": "Technology"},
        {"Company": "Atlassian", "industry": "Technology"},
        {"Company": "Leidos", "industry": "Technology"},
        {"Company": "Duolingo", "industry": "Technology"},
        {"Company": "Logitech", "industry": "Technology"},
        {"Company": "Celestica", "industry": "Technology"},
        {"Company": "Dropbox", "industry": "Technology"},
        {"Company": "Plexus", "industry": "Technology"},
        {"Company": "Silicon Laboratories", "industry": "Technology"},
        {"Company": "Mobix Labs", "industry": "Technology"},
        {"Company": "Mariott", "industry": "Hospitality"},
        {"Company": "InterContinental Hotels Group", "industry": "Hospitality"},
        {"Company": "Sonder Holdings", "industry": "Hospitality"},
        {"Company": "Hyatt Hotels", "industry": "Hospitality"},
        {"Company": "Royal Caribbean Cruises", "industry": "Hospitality"},
        {"Company": "UnitedHealth", "industry": "Healthcare"},
        {"Company": "Johnson & Johnson", "industry": "Healthcare"},
        {"Company": "AbbVie", "industry": "Healthcare"},
        {"Company": "Novo Nordisk", "industry": "Healthcare"},
        {"Company": "Abbott Laboratories", "industry": "Healthcare"},
        {"Company": "AstraZeneca", "industry": "Healthcare"},
        {"Company": "Merck & Co", "industry": "Healthcare"},
        {"Company": "Intuitive Surgical", "industry": "Healthcare"},
        {"Company": "Medtronic", "industry": "Healthcare"},
        {"Company": "Zoetis", "industry": "Healthcare"},
        {"Company": "Humana", "industry": "Healthcare"},
        {"Company": "Illumina", "industry": "Healthcare"},
        {"Company": "Guardant Health", "industry": "Healthcare"},
        {"Company": "Rhythm Pharmaceuticals", "industry": "Healthcare"},
        {"Company": "Amedisys", "industry": "Healthcare"},
        {"Company": "Rivian Automotive", "industry": "Automotive"},
        {"Company": "Fordy", "industry": "Automotive"},
        {"Company": "lululemon athletica", "industry": "Retail"},
        {"Company": "DICK'S Sporting Goods", "industry": "Retail"},
        {"Company": "GameStop Corp", "industry": "Retail"},
        {"Company": "Texas Roadhouse", "industry": "Retail"},
        {"Company": "Hasbro", "industry": "Retail"},
        {"Company": "Mattel", "industry": "Retail"},
        {"Company": "Wayfair", "industry": "Retail"},
        {"Company": "Peloton", "industry": "Retail"},
        {"Company": "Sally Beauty", "industry": "Retail"},
        {"Company": "Lifetime Brand", "industry": "Retail"},
        {"Company": "Allbirds", "industry": "Retail"},
        {"Company": "Walmart", "industry": "Retail"},
        {"Company": "Tyson Foods", "industry": "Retail"},
        {"Company": "Sprouts Farmers Market", "industry": "Retail"},
        {"Company": "Dollar Tree", "industry": "Retail"},
        {"Company": "Stride", "industry": "Retail"},
        {"Company": "Spectrum Brands", "industry": "Retail"},
        {"Company": "Udemy", "industry": "Retail"},
        {"Company": "Vital Farms", "industry": "Retail"},
        {"Company": "Graham Holdings Company", "industry": "Retail"},
        {"Company": "Hims & Hers Health", "industry": "Retail"},
        {"Company": "Smithfield Foods", "industry": "Retail"},
        {"Company": "Albertsons Companies", "industry": "Retail"},
        {"Company": "Albany International", "industry": "Manufacturing"},
        {"Company": "IT Tech Packaging", "industry": "Manufacturing"},
        {"Company": "Lockheed Martin Corporation", "industry": "Manufacturing"},
        {"Company": "Landstar System", "industry": "Manufacturing"},
        {"Company": "Hexcel Corporation", "industry": "Manufacturing"},
        {"Company": "AeroVironment", "industry": "Manufacturing"},
        {"Company": "Matson", "industry": "Manufacturing"},
        {"Company": "McGrath RentCorp", "industry": "Manufacturing"},
        {"Company": "Mueller Industries", "industry": "Manufacturing"},
        {"Company": "Dolby Laboratories", "industry": "Manufacturing"},
        {"Company": "ManpowerGroup", "industry": "Manufacturing"},
        {"Company": "Welltower", "industry": "RealEstate"},
        {"Company": "Iron Mountain Incorporated", "industry": "RealEstate"},
        {"Company": "Camden Property", "industry": "RealEstate"},
        {"Company": "CubeSmart", "industry": "RealEstate"},
        {"Company": "Federal Realty Investment Trust", "industry": "RealEstate"},
        {"Company": "Essential Properties Realty", "industry": "RealEstate"},
        {"Company": "Compass", "industry": "RealEstate"},
        {"Company": "Medical Properties Trust", "industry": "RealEstate"},
        {"Company": "Broadstone", "industry": "RealEstate"},
        {"Company": "Ladder Capital Corp", "industry": "RealEstate"},
        {"Company": "Peakstone Realty Trus", "industry": "RealEstate"},
        {"Company": "Fathom Holdings", "industry": "RealEstate"},
        {"Company": "Presidio Property Trust", "industry": "RealEstate"},
        {"Company": "Service Properties Trust", "industry": "RealEstate"},
    ]

stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'NVDA', 'META', 'JPM', 'V', 'UNH']
bonds = ['US10Y', 'US30Y', 'CORP1', 'CORP2', 'MUNI1', 'MUNI2']
mutual_funds = ['VFIAX', 'SWPPX', 'FXAIX', 'VTSAX', 'FZROX', 'SPY']
# All symbols with types
instruments = (
    [(sym, 'Stock') for sym in stocks] +
    [(sym, 'Bond') for sym in bonds] +
    [(sym, 'Mutual Fund') for sym in mutual_funds]
)

# Generate synthetic data for the tables
employees_df = generate_employee_data(num_employees=100, ourcompany_name = 'bankwell')
accounts_df = generate_account_data(num_accounts=1000, clients_company = clients_company_info)
opportunities_df = generate_opportunity_data(accounts_df, num_opportunities_per_account=3)
contacts_df = generate_employee_contact_data(employees_df, accounts_df, num_relationships_per_employee=30)
portfolio_df = generate_portfolios(accounts_df, max_positions=10, instruments = instruments)

# Print the first few rows of each DataFrame
print("Employees Data:")
print(employees_df.head())

print("\nAccounts Data:")
print(accounts_df.head())

print("\nOpportunities Data:")
print(opportunities_df.head())

print("\nEmployee-Contact Relationships Data:")
print(contacts_df.head())

print("\nPortfolio Info for Each Account")
print(portfolio_df.head())





Employees Data:
                             EmployeeID FirstName    LastName  \
0  389300f0-5cac-4a60-ac26-e0d633027e4f      Jane  Moneypenny   
1  4e87f0b4-7467-40ab-90bb-66eabd26b260   Michael      Harmon   
2  e27b912b-579e-43b3-b665-26b4ca809a2c       Joe      Thomas   
3  e5907e26-1a3b-4b4d-bf29-b5ad343f7999     Bryan        Mann   
4  fd584f2d-a377-4968-ad58-34f60d97d88e  Jonathan       Mason   

                         Title                          Email  \
0         Relationship Manager  jane.moneypenny1@bankwell.com   
1  Senior Relationship Manager    michael.harmon@bankwell.com   
2           Product Specialist        joe.thomas@bankwell.com   
3  Senior Relationship Manager        bryan.mann@bankwell.com   
4  Senior Relationship Manager    jonathan.mason@bankwell.com   

                 Phone    HireDate             Department         Market  
0         214.526.1592  2022-10-06  Enterprise Investment  San Fransisco  
1     749-311-8543x665  2023-04-24             Opera

# Load to database

In [44]:
employees_df.head()

,EmployeeID,FirstName,LastName,Title,Email,Phone,HireDate,Department,Market
0,389300f0-5cac-4a60-ac26-e0d633027e4f,Jane,Moneypenny,Relationship Manager,jane.moneypenny1@bankwell.com,214.526.1592,2022-10-06,Enterprise Investment,San Fransisco
1,4e87f0b4-7467-40ab-90bb-66eabd26b260,Michael,Harmon,Senior Relationship Manager,michael.harmon@bankwell.com,749-311-8543x665,2023-04-24,Operations,San Fransisco
2,e27b912b-579e-43b3-b665-26b4ca809a2c,Joe,Thomas,Product Specialist,joe.thomas@bankwell.com,(291)905-8442x83752,2024-04-06,Customer Support,New York City
3,e5907e26-1a3b-4b4d-bf29-b5ad343f7999,Bryan,Mann,Senior Relationship Manager,bryan.mann@bankwell.com,631-822-0832x1262,2022-05-21,Operations,San Fransisco
4,fd584f2d-a377-4968-ad58-34f60d97d88e,Jonathan,Mason,Senior Relationship Manager,jonathan.mason@bankwell.com,(809)862-5609,2022-11-26,Enterprise Investment,Washington DC


In [45]:
employees_df.shape

(100, 9)

In [46]:
employees_df.to_sql('employees', engine, if_exists='replace', index=False)

100

In [47]:
df = pd.read_sql("SELECT * from Employees", con=engine)
df.head()

,EmployeeID,FirstName,LastName,Title,Email,Phone,HireDate,Department,Market
0,389300f0-5cac-4a60-ac26-e0d633027e4f,Jane,Moneypenny,Relationship Manager,jane.moneypenny1@bankwell.com,214.526.1592,2022-10-06,Enterprise Investment,San Fransisco
1,4e87f0b4-7467-40ab-90bb-66eabd26b260,Michael,Harmon,Senior Relationship Manager,michael.harmon@bankwell.com,749-311-8543x665,2023-04-24,Operations,San Fransisco
2,e27b912b-579e-43b3-b665-26b4ca809a2c,Joe,Thomas,Product Specialist,joe.thomas@bankwell.com,(291)905-8442x83752,2024-04-06,Customer Support,New York City
3,e5907e26-1a3b-4b4d-bf29-b5ad343f7999,Bryan,Mann,Senior Relationship Manager,bryan.mann@bankwell.com,631-822-0832x1262,2022-05-21,Operations,San Fransisco
4,fd584f2d-a377-4968-ad58-34f60d97d88e,Jonathan,Mason,Senior Relationship Manager,jonathan.mason@bankwell.com,(809)862-5609,2022-11-26,Enterprise Investment,Washington DC


In [49]:
accounts_df.head()

,Company,industry,AccountID,ContactFirstName,ContactLastName,ContactTitle,Phone,Email,Website,Location,AnnualRevenue,TotalAccountBal
0,Broadcom,Technology,aa02df83-5f4c-4e5c-8cc5-90d86000d8ca,Sara,Mcpherson,Manager,+1-835-914-7487x34657,sara.mcpherson@broadcom.com,http://www.herrera.com/,Chicago,10142208,9473794
1,Cisco,Technology,b734ebf2-fabb-4a23-ad23-15d458a29e25,Jennifer,Ortiz,Senior Director,001-737-564-1778x6699,jennifer.ortiz@cisco.com,https://tucker.com/,San Fransisco,18929708,16662108
2,Palantir Technologies,Technology,6ceb2a60-83b3-4370-8025-59b18322aab9,Mary,Ramsey,Consultant,843-620-6427,mary.ramsey@palantirtechnologies.com,https://www.johnson.info/,Dallas,30548791,3116148
3,Fiserv,Technology,492cde07-8a6c-4d77-81d1-1363ee54d530,Michael,Davis,Director,001-612-961-3897x025,michael.davis@fiserv.com,https://www.munoz.com/,Los Angeles,34327395,22953196
4,Atlassian,Technology,5cc28d20-f565-43f6-a2b7-6fa5a8ad3469,Betty,Norton,VP,(230)619-7087x8276,betty.norton@atlassian.com,https://davis.info/,Dallas,39968395,13446949


In [50]:
accounts_df.shape

(83, 12)

In [51]:
accounts_df.to_sql('clients_contact', engine, if_exists='replace', index=False)

83

In [52]:
contacts_df.head()

,EmployeeID,EmployeeFirstName,EmployeeLastName,AccountID,Company,Industry,ContactFirstName,ContactLastName,ContactEmail,ContactTitle,ContactPhone
0,389300f0-5cac-4a60-ac26-e0d633027e4f,Jane,Moneypenny,14250a78-8654-4a2f-9c51-e1c6ac33f64d,Rivian Automotive,Automotive,Andrea,Wu,andrea.wu@rivianautomotive.com,VP,284.686.8344x5973
1,389300f0-5cac-4a60-ac26-e0d633027e4f,Jane,Moneypenny,1d98879c-1f8b-4468-97dd-980a5d8505d5,ManpowerGroup,Manufacturing,David,Moreno,david.moreno@manpowergroup.com,Director,001-945-497-7155x003
2,389300f0-5cac-4a60-ac26-e0d633027e4f,Jane,Moneypenny,795d9f43-308a-48bb-b30d-dac589f1da66,Matson,Manufacturing,Anna,Lawrence,anna.lawrence@matson.com,Director,+1-675-665-6673x605
3,389300f0-5cac-4a60-ac26-e0d633027e4f,Jane,Moneypenny,d17b4a50-0ada-4c0f-bcc9-69c11229be12,Sonder Holdings,Hospitality,Shelly,Brooks,shelly.brooks@sonderholdings.com,VP,001-426-351-7808x5323
4,389300f0-5cac-4a60-ac26-e0d633027e4f,Jane,Moneypenny,512a0c94-61c3-4449-b782-3a32417dd182,Walmart,Retail,Karen,Riggs,karen.riggs@walmart.com,Director,+1-246-593-8732x9020


In [60]:
contacts_df.to_sql('client_alignment', engine, if_exists='replace', index=False)

669

In [55]:
portfolio_df.head()

,AccountID,Company_name,symbol,instrument_type
0,aa02df83-5f4c-4e5c-8cc5-90d86000d8ca,Broadcom,US10Y,Bond
1,aa02df83-5f4c-4e5c-8cc5-90d86000d8ca,Broadcom,AAPL,Stock
2,aa02df83-5f4c-4e5c-8cc5-90d86000d8ca,Broadcom,MUNI1,Bond
3,aa02df83-5f4c-4e5c-8cc5-90d86000d8ca,Broadcom,FXAIX,Mutual Fund
4,aa02df83-5f4c-4e5c-8cc5-90d86000d8ca,Broadcom,MSFT,Stock


In [56]:
portfolio_df.to_sql('portfolio', engine, if_exists='replace', index=False)

631

# Test Results

In [58]:
pd.read_sql("SELECT * FROM portfolio limit 10", con=engine)

,AccountID,Company_name,symbol,instrument_type
0,aa02df83-5f4c-4e5c-8cc5-90d86000d8ca,Broadcom,US10Y,Bond
1,aa02df83-5f4c-4e5c-8cc5-90d86000d8ca,Broadcom,AAPL,Stock
2,aa02df83-5f4c-4e5c-8cc5-90d86000d8ca,Broadcom,MUNI1,Bond
3,aa02df83-5f4c-4e5c-8cc5-90d86000d8ca,Broadcom,FXAIX,Mutual Fund
4,aa02df83-5f4c-4e5c-8cc5-90d86000d8ca,Broadcom,MSFT,Stock
5,aa02df83-5f4c-4e5c-8cc5-90d86000d8ca,Broadcom,GOOGL,Stock
6,b734ebf2-fabb-4a23-ad23-15d458a29e25,Cisco,MUNI1,Bond
7,b734ebf2-fabb-4a23-ad23-15d458a29e25,Cisco,GOOGL,Stock
8,b734ebf2-fabb-4a23-ad23-15d458a29e25,Cisco,MUNI2,Bond
9,b734ebf2-fabb-4a23-ad23-15d458a29e25,Cisco,META,Stock


In [62]:
pd.read_sql("SELECT * FROM client_alignment limit 10", con=engine)

,EmployeeID,EmployeeFirstName,EmployeeLastName,AccountID,Company,Industry,ContactFirstName,ContactLastName,ContactEmail,ContactTitle,ContactPhone
0,389300f0-5cac-4a60-ac26-e0d633027e4f,Jane,Moneypenny,14250a78-8654-4a2f-9c51-e1c6ac33f64d,Rivian Automotive,Automotive,Andrea,Wu,andrea.wu@rivianautomotive.com,VP,284.686.8344x5973
1,389300f0-5cac-4a60-ac26-e0d633027e4f,Jane,Moneypenny,1d98879c-1f8b-4468-97dd-980a5d8505d5,ManpowerGroup,Manufacturing,David,Moreno,david.moreno@manpowergroup.com,Director,001-945-497-7155x003
2,389300f0-5cac-4a60-ac26-e0d633027e4f,Jane,Moneypenny,795d9f43-308a-48bb-b30d-dac589f1da66,Matson,Manufacturing,Anna,Lawrence,anna.lawrence@matson.com,Director,+1-675-665-6673x605
3,389300f0-5cac-4a60-ac26-e0d633027e4f,Jane,Moneypenny,d17b4a50-0ada-4c0f-bcc9-69c11229be12,Sonder Holdings,Hospitality,Shelly,Brooks,shelly.brooks@sonderholdings.com,VP,001-426-351-7808x5323
4,389300f0-5cac-4a60-ac26-e0d633027e4f,Jane,Moneypenny,512a0c94-61c3-4449-b782-3a32417dd182,Walmart,Retail,Karen,Riggs,karen.riggs@walmart.com,Director,+1-246-593-8732x9020
5,389300f0-5cac-4a60-ac26-e0d633027e4f,Jane,Moneypenny,1d98879c-1f8b-4468-97dd-980a5d8505d5,ManpowerGroup,Manufacturing,David,Moreno,david.moreno@manpowergroup.com,Director,001-945-497-7155x003
6,389300f0-5cac-4a60-ac26-e0d633027e4f,Jane,Moneypenny,795d9f43-308a-48bb-b30d-dac589f1da66,Matson,Manufacturing,Anna,Lawrence,anna.lawrence@matson.com,Director,+1-675-665-6673x605
7,389300f0-5cac-4a60-ac26-e0d633027e4f,Jane,Moneypenny,9a404093-00b5-426f-9518-2da260321ff5,Presidio Property Trust,RealEstate,Kelly,Smith,kelly.smith@presidiopropertytrust.com,Senior Director,001-671-821-6029
8,389300f0-5cac-4a60-ac26-e0d633027e4f,Jane,Moneypenny,9a404093-00b5-426f-9518-2da260321ff5,Presidio Property Trust,RealEstate,Kelly,Smith,kelly.smith@presidiopropertytrust.com,Senior Director,001-671-821-6029
9,389300f0-5cac-4a60-ac26-e0d633027e4f,Jane,Moneypenny,f841a12f-43dc-44c5-953a-a78bb2050cbf,Silicon Laboratories,Technology,Courtney,Sweeney,courtney.sweeney@siliconlaboratories.com,Consultant,(221)467-4403


In [57]:
import datetime
import pytz

print(f"Notebook last execution time: {datetime.datetime.now(pytz.timezone('US/Central')).strftime('%a, %d %B %Y %H:%M:%S')}")

Notebook last execution time: Thu, 24 April 2025 19:13:07
